### 유튜브 비디오 자막 정보 활용

In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
from types import SimpleNamespace
from youtube_transcript_api.formatters import TextFormatter
import textwrap

def get_video_id(video_url):
    video_id = video_url.split('v=')[1][:11]
    return video_id

video_url = "https://www.youtube.com/watch?v=pSJrML-TTmI"
video_id = get_video_id(video_url)

transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['ko'])
transcript_objects = [SimpleNamespace(**item) for item in transcript]

text_formatter = TextFormatter()
text_formatted = text_formatter.format_transcript(transcript_objects)
text_info = text_formatted.replace("\n", " ")

shorten_text_info = textwrap.shorten(text_info, 150 ,placeholder=' [..이하 생략..]')
print(shorten_text_info) # 축약 출력
# print(text_info) # 전체 내용 출력

이 사람 덕분에 어렵고 복잡하다고만 생각했던 ‘수학'을 다시 보게 됐다는 분들 많습니다. 수학자에게 주어지는 큰 영예죠. 필즈상을 받은 허준이 프린스턴대 교수가 지난 주말 시상식을 마치고 귀국했고, 오늘 KBS에 직접 나왔습니다. 빨리 만나보죠. [..이하 생략..]


In [2]:
import openai

def answer_from_given_info(question_info, prompt):

    # 입력 정보에 기반해 답변 요청
    user_content = f"{prompt} 다음 내용을 바탕으로 질문에 답해 줘. {question_info}" 
    
    messages = [ {'role': 'user', 'content': user_content} ]

    response = openai.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = messages,                        
        max_tokens = 500,
        stop = ["."],
        temperature = 0.8
    )
    
    return response.choices[0].message.content # 응답 결과 반환

In [3]:
question_info = text_info # 자막을 가져온 내용을 학습 데이터로 사용
prompt = "허준이 교수가 받은 상은 무엇인가요?" # 질문
response = answer_from_given_info(question_info, prompt)
print(response)

허준이 교수가 받은 상은 필즈상입니다


In [4]:
question_info = text_info # 자막을 가져온 내용을 학습 데이터로 사용
prompt = "허준이 교수는 어느 대학 교수인가요?" # 질문
response = answer_from_given_info(question_info, prompt)
print(response)

허준이 교수는 프린스턴대학교 교수입니다


### 음성에서 추출한 정보 활용

In [5]:
import yt_dlp
from pathlib import Path

# 유튜브 비디오 정보를 가져오는 함수
def get_youtube_video_info(video_url):
    ydl_opts = {
        'noplaylist': True,
        'quiet': True,
        'no_warnings': True,
    }
    
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        video_info = ydl.extract_info(video_url, download=False) # 비디오 정보 추출
        video_id = video_info['id']              # 비디오 정보에서 비디오 ID 추출
        title = video_info['title']              # 비디오 정보에서 제목 추출
        upload_date = video_info['upload_date']  # 비디오 정보에서 업로드 날짜 추출
        channel = video_info['channel']          # 비디오 정보에서 채널 이름 추출
        duration = video_info['duration_string'] # 비디오 정보에서 영상 길이 추출

    return video_id, title, upload_date, channel, duration

# 파일 이름에 부적합한 문자를 제거하는 함수
def remove_invalid_char_for_filename(input_str):
    # 윈도우 파일 이름에 안 쓰는 문자 제거 
    invalid_characters = '<>:"/\\|?*'
    
    for char in invalid_characters:
        input_str = input_str.replace(char, '_')
        
    # 파일명 마지막에 . 제거
    while input_str.endswith('.'):
        input_str = input_str[:-1]  
        
    return input_str

# 유튜브 비디오를 오디오 파일로 다운로드하는 함수 
def download_youtube_as_mp3(video_url, folder, file_name=None):
    
    _, title, _, _, _ = get_youtube_video_info(video_url)
    filename_no_ext = remove_invalid_char_for_filename(title)
        
    if file_name == None:
        download_file = f"{filename_no_ext}.mp3"
    else:
        download_file = file_name

    outtmpl_str = f'{folder}/{download_file}'
    download_path = Path(outtmpl_str)  
        
    ydl_opts = {     
        'extract_audio': True,      # 다양한 옵션 지정                    
        'format': 'bestaudio/best', # 다운로드 형식 지정 (최적)
        'outtmpl': outtmpl_str,     # 다운로드 경로 지정
        'noplaylist': True,
        'quiet': True,
        'no_warnings': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        video_info = ydl.extract_info(video_url, download=False) # 비디오 정보 추출
        title = video_info.get('title', None) # 비디오 정보 중 제목만 추출
        ydl.download(video_url) # 다운로드

    return title, download_path

In [6]:
video_url = "https://www.youtube.com/watch?v=EKqQvzyVAh4" # 한국어와 영어 모두 있는 비디오
download_folder = "download" # 다운로드 폴더

# 유튜브 비디오를 오디오 파일로 다운로드
file_name = "youtube_video_KBS_news.mp3"
title, download_path = download_youtube_as_mp3(video_url, download_folder, file_name)

print("- 유튜브 제목:", title)
print("- 다운로드한 파일명:", download_path.name)

- 유튜브 제목: “AI 개발, 멈출 때 아냐…갤럭시 ‘빙’ 탑재, 삼성에 달려” [9시 뉴스] / KBS  2023.04.18.
- 다운로드한 파일명: youtube_video_KBS_news.mp3


In [ ]:
from pathlib import Path
from openai import OpenAI
import textwrap

def audio_transcribe(input_path, resp_format="text", lang="en"):
    client = OpenAI()

    with open(input_path, "rb") as f: # 음성 파일 열기
        # 음성 추출  
        transcript_response = client.audio.transcriptions.create(
            model = "gpt-4o-mini-transcribe",
            file = f,
            language = lang,            
            response_format = resp_format # 추출할 텍스트 형식 지정
        )
    
    # 음성을 텍스트로 추출한 후에 텍스트 파일로 저장
    path = Path(input_path)
    output_file = path.with_suffix('.txt') if resp_format == "text" else path.with_suffix('.srt')
        
    with open(output_file, 'w', encoding='utf-8') as f:
        if isinstance(transcript_response, str):
            f.write(transcript_response)
        else:
            f.write(transcript_response.text)

    return transcript_response, output_file

audio_path = download_path
   
print(f"[오디오 파일명] {audio_path.name}\n")

transcript, output_file = audio_transcribe(audio_path)
shorten_text = textwrap.shorten(transcript, 250, placeholder=' [..이하 생략..]')

print(f"- [출력 파일] {output_file.name}")
print(f"- [음성 추출 결과]\n{shorten_text}" )

In [ ]:
input_file = output_file
translated_file = transplate_text_file(input_file)

print("- 번역 파일:", translated_file.name)

In [ ]:
file_name = translated_file # 유튜브 내용을 학습 데이터로 사용
with open(file_name, encoding='utf-8') as f: # 텍스트 파일 읽기
    text = f.read() # 텍스트 파일 내용

question_info = text # 질문에 대한 학습 데이터
prompt = "마이크로소프트는 OpenAI 개발에 얼마를 투자했나요?" # 질문

response = answer_from_given_info(question_info, prompt)
print(response)

In [ ]:
prompt = "KSB가 인터뷰한 사람은 누구인가요?"
response = answer_from_given_info(question_info, prompt)
print(response)